In [1]:
#Libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

In [2]:
#Functions from .py
import src.clean as cl

In [3]:
cd Input

/Users/nicostan/Downloads/Ironhack/labs/project-II/Input


In [4]:
#creating the DFs
fertility = pd.read_csv ('Fertility.csv', encoding='latin')
gdp = pd.read_csv ('GDP per capita.csv', encoding='latin')
life = pd.read_csv ('Life expectancy.csv', encoding='latin')
meat = pd.read_csv ('Meat consumption.csv', encoding='latin')
age = pd.read_csv ('Median age.csv', encoding='latin')
growth = pd.read_csv ('Population growth.csv', encoding='latin')
sex = pd.read_csv ('Sex-ratio.csv', encoding='latin')
suicide = pd.read_csv ('Suicide rate.csv', encoding='latin')
urbanization = pd.read_csv ('Urbanization rate.csv', encoding='latin')

In [5]:
cd ..

/Users/nicostan/Downloads/Ironhack/labs/project-II


In [6]:
#Checking for missing countries
print(f"fertility has {fertility.shape[0]} rows")
print(f"gdp has {gdp.shape[0]} rows")
print(f"life has {life.shape[0]} rows")
print(f"meat has {meat.shape[0]} rows")
print(f"age has {age.shape[0]} rows")
print(f"growth has {growth.shape[0]} rows")
print(f"sex has {sex.shape[0]} rows")
print(f"suicide has {suicide.shape[0]} rows")
print(f"urbanization has {urbanization.shape[0]} rows")

fertility has 201 rows
gdp has 191 rows
life has 185 rows
meat has 187 rows
age has 223 rows
growth has 207 rows
sex has 226 rows
suicide has 182 rows
urbanization has 218 rows


In [7]:
#Dropping country columns for future merge, we don't want to drop sex because it has the most countries(rows)
fertility = cl.dropping(fertility,'Country')
gdp = cl.dropping(gdp,'Country')
life = cl.dropping(life,'Country')
meat = cl.dropping(meat,'Country')
age = cl.dropping(age,'Country')
growth = cl.dropping(growth,'Country')
suicide = cl.dropping(suicide,'Country')
urbanization = cl.dropping(urbanization,'Country')

In [8]:
#Quick fix for formatting meat, sex and age df, also for solving the joining issue ahead
sex = cl.formatting(sex,'Country', 'ISO-code')
fertility = cl.formatting(fertility,'ISO-code')
gdp = cl.formatting(gdp,'ISO-code')
life = cl.formatting(life,'ISO-code')
meat = cl.formatting(meat,'ISO-code')
age = cl.formatting(age,'ISO-code')
growth = cl.formatting(growth,'ISO-code')
suicide = cl.formatting(suicide,'ISO-code')
urbanization = cl.formatting(urbanization,'ISO-code')

In [9]:
#Only unique values
fertility = fertility.drop_duplicates(subset='ISO-code')
gdp = gdp.drop_duplicates(subset='ISO-code')
life = life.drop_duplicates(subset='ISO-code')
growth = growth.drop_duplicates(subset='ISO-code')
suicide = suicide.drop_duplicates(subset='ISO-code')
urbanization = urbanization.drop_duplicates(subset='ISO-code')
meat = meat.drop_duplicates(subset='ISO-code')
sex = sex.drop_duplicates(subset='ISO-code')
age = age.drop_duplicates(subset='ISO-code')

In [10]:
#Add missing countries on sex for final considerations and limitations
sex = sex.append({'Country':'Palestine', 'ISO-code':'PSE'}, ignore_index=True)
sex = sex.append({'Country':'Martinique', 'ISO-code':'MTQ'}, ignore_index=True)
sex = sex.append({'Country':'Guadeloupe', 'ISO-code':'GLP'}, ignore_index=True)
sex = sex.append({'Country':'French Guiana', 'ISO-code':'GUF'}, ignore_index=True)
sex = sex.append({'Country':'Niue', 'ISO-code':'NIU'}, ignore_index=True)
sex = sex.append({'Country':'Réunion', 'ISO-code':'REU'}, ignore_index=True)
sex = sex.append({'Country':'Guinea-Bissau', 'ISO-code':'GBS'}, ignore_index=True)
sex = sex.append({'Country':'Mayotte', 'ISO-code':'MAY'}, ignore_index=True)

/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/53380816.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sex = sex.append({'Country':'Palestine', 'ISO-code':'PSE'}, ignore_index=True)
/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/53380816.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sex = sex.append({'Country':'Martinique', 'ISO-code':'MTQ'}, ignore_index=True)
/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/53380816.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sex = sex.append({'Country':'Guadeloupe', 'ISO-code':'GLP'}, ignore_index=True)
/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/53380816.py:5: FutureWarning: The frame.append meth

In [11]:
#Left join on sex as it is the most complete DF --> Consolidating the DF (df)
df = sex.merge(fertility, on=['ISO-code'], how='left')
df = df.merge(gdp, on=['ISO-code'], how='left')
df = df.merge(life, on=['ISO-code'], how='left')
df = df.merge(meat, on=['ISO-code'], how='left')
df = df.merge(age, on=['ISO-code'], how='left')
df = df.merge(growth, on=['ISO-code'], how='left')
df = df.merge(suicide, on=['ISO-code'], how='left')
df = df.merge(urbanization, on=['ISO-code'], how='left')
df = cl.dropping(df,'ISO-code')
df.shape[0]

232

In [12]:
df.sample()

,Country,Sex-ratio,Fertility,GDP per capita,Life expectancy,Meat consumption,Median age,Population growth,Suicide rate,Urbanization rate
3,American Samoa,1.0,NaN,NaN,NaN,24.9,25.5,-0.26,NaN,87.2


In [13]:
#Scraping migration data from wikipedia
url = "https://en.wikipedia.org/wiki/List_of_countries_by_net_migration_rate"
res = requests.get(url)
html=res.content
soup = BeautifulSoup(html, "html.parser")

In [14]:
#The second table is the charm
tables = soup.find_all("table")[1] 

In [15]:
#Make the DF (df2) and make it look pretty
df2 = pd.read_html(tables.prettify())[0]
df2.rename(columns = {'Annual net migration per 1000 inhabitants  2010–2015':'Net Migration'}, inplace = True)
df2=cl.dropping(df2,'Annual net migration per 1000 inhabitants  2015–2020 (forecast)')
df = cl.formatting(df,'Country')
df2 = cl.formatting(df2,'Country')
list(df2.keys())

['Country', 'Net Migration']

In [16]:
#Correcting country names for an accurate Merge
df2['Country'][13]='Curacao'
df2['Country'][28]='Jersey'
df2 = df2.append({'Country':'Guernsey', 'Net Migration':df2['Net Migration'][28]}, ignore_index=True)
df2['Country'][31]='United Kingdom'
df2['Country'][37]='The Bahamas'
df2['Country'][73]='Democratic Republic of the Congo'
df2['Country'][119]='The Gambia'
df2['Country'][138]='Republic of the Congo'
df2['Country'][179]='Virgin Islands'
df2['Country'][180]='Sao Tome and Principe'
df2['Country'][189]='Samoa'

/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/2641628015.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'Country':'Guernsey', 'Net Migration':df2['Net Migration'][28]}, ignore_index=True)


In [17]:
##Left join on df to consolidate the DF (df <--df2)
df = df.merge(df2, on=['Country'], how='left')
df.shape[0]

232

In [18]:
df.sample()

,Country,Sex-ratio,Fertility,GDP per capita,Life expectancy,Meat consumption,Median age,Population growth,Suicide rate,Urbanization rate,Net Migration
211,United Kingdom,0.99,1.7,48169.0,81.2,79.6,40.5,0.75,7.6,83.9,4.0


In [19]:
#Adding the Continent column
df3 = pd.read_csv('Input/csvData.csv')
df3.rename(columns = {'country':'Country', 'continent':'Continent'}, inplace = True)
df3 = cl.formatting(df3,'Country')
df3.sample()

,Country,Continent
29,Malawi,Africa


In [20]:
#Correcting country names for an accurate Merge
df3["Country"][160]='The Bahamas'
df3 = df3.append({'Country':'Democratic Republic of the Congo', 'Continent':'Africa'}, ignore_index=True)
df3["Country"][19]='The Gambia'
df3["Country"][100]='East Timor'
df3.drop_duplicates(subset='Country', ignore_index=True, inplace=True)

/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/674111837.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append({'Country':'Democratic Republic of the Congo', 'Continent':'Africa'}, ignore_index=True)


In [21]:
#Export the file continents for a cleaner look
df3.to_csv("./Input/continents.csv")

In [22]:
##Left join on df to consolidate the DF (df <--df3)
df = df.merge(df3, on=['Country'], how='left')
df.sample()

,Country,Sex-ratio,Fertility,GDP per capita,Life expectancy,Meat consumption,Median age,Population growth,Suicide rate,Urbanization rate,Net Migration,Continent
178,Seychelles,1.07,2.4,33118.0,73.3,51.1,35.4,0.39,8.3,57.5,−2.1,Africa


In [23]:
#Countries without continent --> Small Island Nations which will be dropped on further analysis
df['Continent'].isna().sum()

4

In [24]:
#Importing data on Developped / In development
first = pd.read_csv('Input/first.csv')
third = pd.read_csv('Input/third.csv')

In [25]:
#Dropping unnecessary columns
first = cl.dropping(first,'hdiRank', 'pop2022')
third = cl.dropping(third,'pop2022')

In [26]:
#Renaming columns for comprehensive reading
first.rename(columns = {'country':'Country','hdi2019':'HDI' }, inplace = True)
third.rename(columns = {'country':'Country', 'humanDevelopmentIndex':'HDI'}, inplace = True)

In [27]:
#Adding the correct collumn for each 'Class' and 
first['Class']='Developped'
third['Class']='In development'
#Correcting country names for an accurate Merge
first["Country"][55]='The Bahamas'
third['Country'][87]='American Samoa'
third['Country'][15]='Democratic Republic of the Congo'
third['Country'][17]='The Gambia'
third['Country'][59]='East Timor'

/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/1264053327.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first["Country"][55]='The Bahamas'
/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/1264053327.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  third['Country'][87]='American Samoa'
/var/folders/5v/dc6514ld10d33dy0zgf23r6m0000gn/T/ipykernel_63541/1264053327.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [28]:
#Concatenating first and third into df4. Dropping duplicates, formatting and sorting A-Z.
df4 = pd.concat([first, third], ignore_index=True)
df4.drop_duplicates(subset='Country', keep='first', inplace=True, ignore_index=True)
df4 = cl.formatting(df4,'Country')
df4=df4.sort_values(by='Country', axis=0, ascending=True, ignore_index=True)
df4.shape[0]

191

In [29]:
#Export the file worlds for a cleaner look
df4.to_csv("./Input/worlds.csv")

In [30]:
##Left join on df to consolidate the DF (df <--df3)
df = df.merge(df4, on=['Country'], how='left')

In [31]:
#Fill NaN values in Class columns as Nations ' In development'
df['Class'].fillna(value='In development', inplace=True)

In [32]:
#Checking for duplicate countries --> None found
df.drop_duplicates(subset='Country', ignore_index=True, inplace=True)
df=df.sort_values(by='Country', axis=0, ascending=True, ignore_index=True)
df.shape[0]

232

In [33]:
#Export the main DF (df) for a cleaner look and further plotting
df.to_csv("./Output/df_clean.csv")